<a href="https://colab.research.google.com/github/justadudewhohacks/ipynbs/blob/master/age_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dependencies


In [0]:
!pip install -U -q PyDrive

### Download Data

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

utk_images_7z_id = ''
utk_landmarks_7z_id = ''

appareal_labels_json_id = ''
appareal_images_7z_id = ''
appareal_landmarks_7z_id = ''

wiki_labels_json_id = ''
wiki_images_7z_id = ''
wiki_landmarks_7z_id = ''

if not os.path.exists('./data'):
  os.makedirs('./data')
if not os.path.exists('./data/utk'):
  os.makedirs('./data/utk')
if not os.path.exists('./data/appareal'):
  os.makedirs('./data/appareal')
if not os.path.exists('./data/wiki'):
  os.makedirs('./data/wiki')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
  
print('downloading utk data...')
drive.CreateFile({ 'id': utk_images_7z_id }).GetContentFile('./data/utk/images.7z')
drive.CreateFile({ 'id': utk_landmarks_7z_id }).GetContentFile('./data/utk/landmarks.7z')

print('downloading appareal data...')
drive.CreateFile({ 'id': appareal_labels_json_id }).GetContentFile('./data/appareal/labels.json')
drive.CreateFile({ 'id': appareal_images_7z_id }).GetContentFile('./data/appareal/images.7z')
drive.CreateFile({ 'id': appareal_landmarks_7z_id }).GetContentFile('./data/appareal/landmarks.7z')

print('downloading wiki data...')
drive.CreateFile({ 'id': wiki_labels_json_id }).GetContentFile('./data/wiki/labels.json')
drive.CreateFile({ 'id': wiki_images_7z_id }).GetContentFile('./data/wiki/images.7z')
drive.CreateFile({ 'id': wiki_landmarks_7z_id }).GetContentFile('./data/wiki/landmarks.7z')
  
print('done!')

### Unzip Data

In [0]:
!cd ./data/utk && p7zip -d ./images.7z
!cd ./data/utk && p7zip -d ./landmarks.7z
!cd ./data/appareal && p7zip -d ./images.7z
!cd ./data/appareal && p7zip -d ./landmarks.7z
!cd ./data/wiki && p7zip -d ./images.7z
!cd ./data/wiki && p7zip -d ./landmarks.7z

### Preprocessing

In [0]:
import cv2
import math
import json
import numpy as np
from random import randint

def num_in_range(val, min_val, max_val):
  return min(max(min_val, val), max_val)

def random_crop(img, landmarks):
  height, width, _ = img.shape
  min_x, min_y, max_x, max_y = width, height, 0, 0
  for pt in landmarks:
    min_x = pt['x'] if pt['x'] < min_x else min_x
    min_y = pt['y'] if pt['y'] < min_y else min_y
    max_x = max_x if pt['x'] < max_x else pt['x']
    max_y = max_y if pt['y'] < max_y else pt['y']
  
  min_x = int(num_in_range(min_x, 0, 1) * width)
  min_y = int(num_in_range(min_y, 0, 1) * height)
  max_x = int(num_in_range(max_x, 0, 1) * width)
  max_y = int(num_in_range(max_y, 0, 1) * height)
  x0 = randint(0, min_x)
  y0 = randint(0, min_y)
  x1 = randint(0, abs(width - max_x)) + max_x
  y1 = randint(0, abs(height - max_y)) + max_y

  return img[y0:y1, x0:x1]

def resize_preserve_aspect_ratio(img, size):
  height, width, _ = img.shape
  max_dim = max(height, width)
  ratio = size / float(max_dim)
  shape = (height * ratio, width * ratio)
  resized_img = cv2.resize(img, (int(round(height * ratio)), int(round(width * ratio))))
  
  return resized_img
  
def pad_to_square(img):
  height, width, channels = img.shape
  max_dim = max(height, width)
  square_img = np.zeros([max_dim, max_dim, channels])

  dx = math.floor(abs(max_dim - width) / 2)
  dy = math.floor(abs(max_dim - height) / 2)
  square_img[dy:dy + height,dx:dx + width] = img

  return square_img

def preprocess(img, landmarks, size):
  cropped_img = random_crop(img, landmarks)
  resized_img = resize_preserve_aspect_ratio(cropped_img, size)
  square_img = pad_to_square(resized_img)
  
  return square_img

def load_batch(datas):
  preprocessed_imgs = []
  
  for data in datas:
    db = data['db']
    img_file = data['file']
    file_suffix = 'chip_0' if db == 'utk' else ('face_0' if db == 'appareal' else '')
    landmarks_file = img_file.replace(file_suffix + '.jpg', file_suffix + '.json')
    img_file_path = './data/' + db + '/cropped-images/' + img_file
    landmarks_file_path = './data/' + db + '/landmarks/' + landmarks_file
    
    img = cv2.imread(img_file_path)
    with open(landmarks_file_path) as json_file:  
      landmarks = json.load(json_file)
      preprocessed_img = preprocess(img, landmarks, 112)
      preprocessed_imgs.append(preprocessed_img)
      
  return np.stack(preprocessed_imgs, axis=0)
  

### Training